Overall Plan:
1. Get the compositions we care about in V-Cr-Ti space
2. Fit CE model on them 
3. Run MCMC on those compositions 
4. Sample 3-5 structures with comparable energy
5. Relax the supercells using CHGNet 
6. Create vacancies in those supercells 
7. Relax the supercell (letting the cell change)
8. Create defects for all nearest neighbors 
9. relax with keeping the volume fixed 
10. interpolate between start and end point
11. conduct neb 
12. save the energies for that site and neighbor 
13. for each structure, create histogram of the mean and std energies over all NNs 

# 1 Get all the compositions we care about in the V-Cr-Ti Space


## Libraries

In [10]:
import os
from pymatgen.core import Structure

## Load the perfect structures

In [11]:
# load the perfect structures 
#start_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/V0_90625-Cr0_046875-Ti0_046875_initial.cif'
#middle_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/V0_90625-Cr0_046875-Ti0_046875_middle.cif'
#end_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_4096/V0_90625-Cr0_046875-Ti0_046875_final.cif'
start_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_512/V0_90625-Cr0_046875-Ti0_046875_initial.cif'
middle_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_512/V0_90625-Cr0_046875-Ti0_046875_middle.cif'
end_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_512/V0_90625-Cr0_046875-Ti0_046875_final.cif'


start = Structure.from_file(start_path)
middle = Structure.from_file(middle_path)
end = Structure.from_file(end_path)

 # 2. Create Vacancies from the Structures

## Libraries

In [21]:
import os
import json
import sys
import numpy as np
import random
from smol.io import load_work
from pymatgen.core.structure import Structure
from pymatgen.entries.computed_entries import ComputedStructureEntry
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.io.vasp.outputs import Outcar
from ase.db import connect
from ase.io import write
from ase.visualize import view

sys.path.append('../Modules')
from defect_maker import make_defects, return_x_neighbors
from vasp_misc import *
# Function to load and sort the structure
def load_and_sort_structure(entry):
    return Structure.from_dict(entry.structure.as_dict()).get_sorted_structure()

def _read_contcar_direct(contcar_file):
    try:
        with open(contcar_file, 'r') as file:
            lines = file.readlines()
            # Read the lattice constant
            lattice_constant = float(lines[1].strip())
            # Read the lattice vectors
            lattice_vectors = [list(map(float, line.strip().split())) for line in lines[2:5]]
            # Read the elements and their counts
            #elements = lines[5].strip().split()
            elements = [element.split('_')[0] for element in lines[5].strip().split()]
            print(elements)
            element_counts = list(map(int, lines[6].strip().split()))
            # Create a list of species that matches the number of coordinates
            species = [element for element, count in zip(elements, element_counts) for _ in range(count)]
            # Read the coordinates
            coordinates = [list(map(float, line.strip().split())) for line in lines[8:8+sum(element_counts)]]
            # Convert the coordinates from direct to Cartesian
            cartesian_coordinates = [[sum(a*b for a, b in zip(coord, vector)) for vector in lattice_vectors] for coord in coordinates]
            # Create the structure
            contcar = Structure(lattice_vectors, species, cartesian_coordinates)
            return contcar
    except Exception as e:
        print(f"Error reading CONTCAR file {contcar_file}: {e}")
        raise e
        return None
    
from pymatgen.core.structure import Structure

def read_contcar_direct(contcar_file):
    try:
        with open(contcar_file, 'r') as file:
            lines = file.readlines()
            # Read the lattice constant
            lattice_constant = float(lines[1].strip())
            # Read the lattice vectors
            lattice_vectors = [list(map(float, line.strip().split())) for line in lines[2:5]]
            # Read the elements and their counts
            elements_line = lines[5].strip().split()
            if '/' in elements_line[0]:
                # Handle format where elements are followed by identifiers
                #elements = [element.split('/')[0] for element in elements_line]
                elements = [element.split('/')[0].rstrip('_pv').rstrip('_sv') for element in elements_line]

            else:
                # Handle format where elements are directly listed
                elements = elements_line
            print(elements)
            element_counts = list(map(int, lines[6].strip().split()))
            # Create a list of species that matches the number of coordinates
            species = [element for element, count in zip(elements, element_counts) for _ in range(count)]
            # coordinate type
            coord_type = lines[7].strip()
            if coord_type.startswith(('c','C')):
                cart = True
            elif coord_type.startswith(('d','D')):
                cart = False
            # Read the coordinates
            coordinates_start_index = 8
            coordinates = []
            for line in lines[coordinates_start_index:coordinates_start_index + sum(element_counts)]:
                parts = line.strip().split()
                coordinates.append(list(map(float, parts[:3])))

            # Convert the coordinates from direct to Cartesian
            #cartesian_coordinates = [
                #[sum(a*b for a, b in zip(coord, vector)) for vector in zip(*lattice_vectors)]
                #for coord in coordinates
            #]
            # Create the structure
            contcar = Structure(lattice_vectors, species, coords=coordinates, coords_are_cartesian=cart)
            return contcar
    except Exception as e:
        print(f"Error reading CONTCAR file {contcar_file}: {e}")
        raise e
        return None


# Function to find N target atoms without overlapping second nearest neighbors
def _find_target_atoms(structure, N, neighbor_distance=2):
    all_indices = list(range(len(structure)))
    random.shuffle(all_indices)
    target_atoms = []
    neighbor_sets = []

    print(f"All indices: {all_indices}")

    while all_indices and len(target_atoms) < N:
        index = all_indices.pop()
        #neighbors, _ = return_x_neighbors(structure, target_atom_index=index, x_neighbor=neighbor_distance, alat=structure.lattice.a)
        neighbors = []
        for distance in range(1, neighbor_distance + 1):
            neighbors_distance, _ = return_x_neighbors(structure, target_atom_index=index, x_neighbor=distance, alat=structure.lattice.a)
            neighbors.extend(neighbors_distance)
        print(f"Index: {index}, Neighbors: {neighbors}")

        if not any(set(neighbors).intersection(neighbor_set) for neighbor_set in neighbor_sets):
            target_atoms.append(index)
            neighbor_sets.append(set(neighbors))
            print(f"Selected target atom index: {index}")

    #return target_atoms if len(target_atoms) == N else None
    return target_atoms 

def find_target_atoms(structure, N, neighbor_distance=2, cutoff_distance=5):
    all_indices = list(range(len(structure)))
    random.shuffle(all_indices)
    target_atoms = []
    neighbor_sets = []

    print(f"All indices: {all_indices}")

    while all_indices and len(target_atoms) < N:
        index = all_indices.pop()
        neighbors = []
        for distance in range(1, neighbor_distance + 1):
            neighbors_distance, _ = return_x_neighbors(structure, target_atom_index=index, x_neighbor=distance, alat=structure.lattice.a)
            neighbors.extend(neighbors_distance)
        
        print(f"Index: {index}, Neighbors: {neighbors}")

        if not any(set(neighbors).intersection(neighbor_set) for neighbor_set in neighbor_sets):
            # Check if the distance to all existing target atoms is greater than the cutoff distance
            if all(structure.get_distance(index, target_atom) > cutoff_distance for target_atom in target_atoms):
                target_atoms.append(index)
                neighbor_sets.append(set(neighbors))
                print(f"Selected target atom index: {index}")

    #return target_atoms if len(target_atoms) == N else None
    return target_atoms

def _randomly_pick_sites(structure, n):
    # Ensure that n is not greater than the number of sites in the structure
    if n > len(structure.sites):
        raise ValueError("The number of sites to pick cannot be greater than the number of sites in the structure.")
    
    # Randomly select n sites from the structure
    random_sites = random.sample(structure.sites, n)
    return random_sites

import random
from scipy.spatial.distance import cdist

def _randomly_pick_sites(structure, n, cutoff=1):
    # Ensure that n is not greater than the number of sites in the structure
    if n > len(structure.sites):
        raise ValueError("The number of sites to pick cannot be greater than the number of sites in the structure.")
    
    # Randomly select n sites from the structure
    random_sites = []
    while len(random_sites) < n:
        potential_site = random.choice(structure.sites)
        print("Potential site: ", potential_site.coords)
        print(cutoff * min(structure.lattice.abc))
        if all(cdist([potential_site.coords], [site.coords])[0][0] > cutoff * min(structure.lattice.abc) for site in random_sites):
            print("Found site")
            random_sites.append(potential_site)
    return random_sites

import random
import numpy as np
from pymatgen.core.structure import Structure

def randomly_pick_sites(structure, n, initial_cutoff=1.25, max_attempts=1000, reduction_factor=0.9):
    """
    Randomly selects a specified number of sites from a given structure.

    Args:
        structure (Structure): The structure from which to randomly select sites.
        n (int): The number of sites to randomly select.
        initial_cutoff (float, optional): The initial cutoff distance for site selection. Defaults to 1.25.
        max_attempts (int, optional): The maximum number of attempts to make for site selection. Defaults to 1000.
        reduction_factor (float, optional): The reduction factor for the cutoff distance after each unsuccessful attempt. Defaults to 0.9.

    Returns:
        list: A list of randomly selected sites from the structure.

    Raises:
        ValueError: If the number of sites to pick is greater than the number of sites in the structure.
    """
    # Ensure that n is not greater than the number of sites in the structure
    if n > len(structure.sites):
        raise ValueError("The number of sites to pick cannot be greater than the number of sites in the structure.")
    
    # Randomly select n sites from the structure with iterative reduction in cutoff
    random_sites = []
    cutoff = initial_cutoff
    while len(random_sites) < n:
        attempts = 0
        while len(random_sites) < n and attempts < max_attempts:
            potential_site = random.choice(structure.sites)
            if all(np.linalg.norm(np.array(potential_site.coords) - np.array(site.coords)) > cutoff * min(structure.lattice.abc) for site in random_sites):
                random_sites.append(potential_site)
            attempts += 1
        
        if len(random_sites) < n:
            cutoff *= reduction_factor
            random_sites = []  # Reset and try again with a reduced cutoff
    
    return random_sites

# Function to select a random neighbor
def select_random_neighbor(structure, target_atom_index, x_neighbor):
    neighbors, _ = return_x_neighbors(structure, target_atom_index, x_neighbor, structure.lattice.a)
    if neighbors:
        return random.choice(neighbors)
    return None

# Function to create and save structures with vacancies
def create_and_save_structures(entries, N, job_path, neighbor_distance=2, cutoff_distance=1.25):
    for k, entry in enumerate(entries):
        print(f"Processing entry {k+1}/{len(entries)}...")
        structure = load_and_sort_structure(entry)
        #target_atoms = find_target_atoms(structure, N, neighbor_distance, cutoff_distance)
        target_atoms = randomly_pick_sites(structure, N, cutoff = cutoff_distance)
        if not target_atoms:
            print(f"No suitable target atoms found for entry {k+1}. Skipping...")
            continue
        print(f"Found target atoms for entry {k+1}: {target_atoms}")

        for n, target_atom_index in enumerate(target_atoms):
            start_structure, _ = make_defects(structure, target_atom_index, target_atom_index)
            print("Start defect made")
            if start_structure is None:
                print(f"Failed to create start structure for entry {k+1}, target atom {target_atom_index}.")
                continue

            for x_neighbor in [1, 2, 3]:
                print("Neigbor distance: ", x_neighbor)
                vac_site = select_random_neighbor(structure, target_atom_index, x_neighbor)
                if vac_site is not None:
                    _, end_structure = make_defects(structure, target_atom_index, vac_site)
                    if end_structure is None:
                        print(f"Failed to create end structure for entry {k+1}, target atom {target_atom_index}, vac_site {vac_site}.")
                        continue

                    # Create directory and filenames
                    directory = os.path.join(job_path, f"structure_{k}_vac_site_{n}")
                    os.makedirs(directory, exist_ok=True)
                    print(f"Created directory: {directory}")

                    start_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_start.vasp")
                    end_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_end_site_{vac_site}.vasp")

                    # Write the structures to POSCAR files
                    Poscar(start_structure).write_file(start_filename)
                    print(f"Written start structure to {start_filename}")
                    Poscar(end_structure).write_file(end_filename)
                    print(f"Written end structure to {end_filename}")

def _create_start_structures(entries, N, job_path, cutoff_distance=1.25):
    start_structures = []
    removed_indexes = []
    for k, entry in enumerate(entries):
        print(f"Processing entry {k+1}/{len(entries)}...")
        structure = load_and_sort_structure(entry)
        target_atoms = randomly_pick_sites(structure, N, cutoff = cutoff_distance)
        if not target_atoms:
            print(f"No suitable target atoms found for entry {k+1}. Skipping...")
            continue
        print(f"Found target atoms for entry {k+1}: {target_atoms}")

        for n, target_atom_index in enumerate(target_atoms):
            start_structure, _ = make_defects(structure, target_atom_index, target_atom_index)
            if start_structure is None:
                print(f"Failed to create start structure for entry {k+1}, target atom {target_atom_index}.")
                continue
            start_structures.append(start_structure)
            removed_indexes.append(target_atom_index)
    return start_structures, removed_indexes

from pymatgen.entries.computed_entries import ComputedStructureEntry
from monty.json import MontyEncoder, MontyDecoder
from pymatgen.io.vasp import Poscar

def create_start_structures(entries, N, job_path, cutoff_distance=1.25,supercell_scheme=True):
    computed_entries = []
    for k, entry in enumerate(entries):
        print(f"Processing entry {k+1}/{len(entries)}...")
        structure = load_and_sort_structure(entry)
        target_atoms = randomly_pick_sites(structure, N, initial_cutoff= cutoff_distance)
        if not target_atoms:
            print(f"No suitable target atoms found for entry {k+1}. Skipping...")
            continue
        print(f"Found target atoms for entry {k+1}: {target_atoms}")

        for n, target_atom_index in enumerate(target_atoms):
            #print("On Target Atom: ", target_atom_index)
            target_atom_index = structure.index(target_atom_index)
            target_atom_composition = structure[target_atom_index].specie

            start_structure, _ = make_defects(structure, target_atom_index, target_atom_index)
            if start_structure is None:
                print(f"Failed to create start structure for entry {k+1}, target atom {target_atom_index}.")
                continue

            # Save the structure as a .vasp file
            #start_filename = os.path.join(job_path, f"structure_{k}_vac_site_{n}_start.vasp")
            if supercell_scheme:
                start_filename = os.path.join(job_path, f"supercell_gen{entry.data['generation']}_comp{entry.data['comp']}_struct{entry.data['struct']}_vac_site{n}_start.vasp")
            else:
                start_filename = os.path.join(job_path, f"supercell_gen{entry.data['generation']}_comp{entry.data['comp']}_vac_site{n}_start.vasp")
            Poscar(start_structure).write_file(start_filename)
            print(f"Written start structure to {start_filename}")

            # Create a ComputedStructureEntry and add it to the list
            data = {'generation': entry.data['generation'],
                    'comp': entry.data['comp'], 
                    'struct': entry.data['struct'], 
                    'vac_index' : target_atom_index, 
                    'vac_comp' : target_atom_composition, 
                    'perfect_structure' : entry.structure.as_dict()}
            computed_entry = ComputedStructureEntry(start_structure, energy=0, data=data)
            computed_entries.append(computed_entry)

    # Save the computed entries as a JSON file
    with open(os.path.join(job_path, 'computed_entries.json'), 'w') as f:
        json.dump(computed_entries, f, cls=MontyEncoder)

    print("Computed entries saved to JSON file.")

def create_end_structures(start_structures, removed_indexes, job_path):
    for k, (start_structure, target_atom_index) in enumerate(zip(start_structures, removed_indexes)):
        for n, x_neighbor in enumerate([1, 2, 3]):
            vac_site = select_random_neighbor(start_structure, target_atom_index, x_neighbor)
            if vac_site is not None:
                _, end_structure = make_defects(start_structure, target_atom_index, vac_site)
                if end_structure is None:
                    print(f"Failed to create end structure for entry {k+1}, target atom {target_atom_index}, vac_site {vac_site}.")
                    continue

                # Create directory and filenames
                directory = os.path.join(job_path, f"structure_{k}_vac_site_{n}")
                os.makedirs(directory, exist_ok=True)
                print(f"Created directory: {directory}")

                start_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_start.vasp")
                end_filename = os.path.join(directory, f"structure_{k}_vac_site_{n}_end_site_{vac_site}.vasp")

                # Write the structures to POSCAR files
                Poscar(start_structure).write_file(start_filename)
                print(f"Written start structure to {start_filename}")
                Poscar(end_structure).write_file(end_filename)
                print(f"Written end structure to {end_filename}")

def check_overlapping_atoms(structure, distance_threshold=0.4):
    """
    Check if a pymatgen structure has overlapping atoms.
    
    Parameters:
    structure (Structure): The pymatgen structure to check.
    distance_threshold (float): The distance threshold below which atoms are considered overlapping.
    
    Returns:
    bool: True if there are overlapping atoms, False otherwise.
    """
    distances = structure.distance_matrix
    num_atoms = len(structure)

    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            if distances[i, j] < distance_threshold:
                return True
    return False

def _print_min_distance(structure: Structure):
    min_distance = float('inf')

    for i in range(len(structure)):
        for j in range(i+1, len(structure)):
            distance = structure[i].distance(structure[j])
            min_distance = min(min_distance, distance)

    print(f"The minimum distance between any two atoms in the structure is: {min_distance}")

def print_min_distance(structure: Structure):
    min_distance = float('inf')
    atom1, atom2 = None, None

    for i in range(len(structure)):
        for j in range(i+1, len(structure)):
            distance = structure[i].distance(structure[j])
            if distance < min_distance:
                min_distance = distance
                atom1, atom2 = i, j

    print(f"The minimum distance between any two atoms in the structure is: {min_distance}")
    print(f"The atoms are at indexes {atom1} and {atom2}")
    print(f"The coordinates of the atoms are {structure[atom1].coords} and {structure[atom2].coords}")

## Create Vacancies

In [61]:
# create an entry from teh structure 
def create_entry(structure, data):
    return ComputedStructureEntry(structure, energy=0, data=data)

# create a list of entries from a list of structures
def create_entries(structures, data):
    return [create_entry(structure, dat) for structure, dat in zip(structures, data)]

# create a list of entries from a list of structures
def create_entries_from_files(structure_files, data):
    structures = [Structure.from_file(file) for file in structure_files]
    data = []
    for structure in structures:
        struc_data = {'generation' : 0, 'comp' : structure.composition.reduced_formula, 'struct' : 'Fixed_64'} 
        data.append(struc_data)
    return create_entries(structures, data)

# get file list
file_list = [os.path.join('../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64',f) for f in os.listdir('../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64/') if f.endswith('.cif')]
#file_list = [start_path, middle_path, end_path]
entries = create_entries_from_files(file_list, {'generation': 0, 'comp': 'VCrTi', 'struct': 'Fixed_64'})

In [72]:
print(entries[6].structure)

Full Formula (Ti3 V58 Cr3)
Reduced Formula: Ti3V58Cr3
abc   :  10.426946  10.426946  10.426946
angles: 109.471221 109.471221 109.471221
pbc   :       True       True       True
Sites (64)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Ti    0     0.25  0.75
  1  Ti    0.75  0     0.25
  2  Ti    0.75  0     0.75
  3  V     0     0     0
  4  V     0     0     0.25
  5  V     0     0     0.5
  6  V     0     0.25  0
  7  V     0     0.25  0.25
  8  V     0     0.25  0.5
  9  V     0     0.5   0
 10  V     0     0.5   0.25
 11  V     0     0.5   0.5
 12  V     0     0.5   0.75
 13  V     0     0.75  0
 14  V     0     0.75  0.25
 15  V     0     0.75  0.5
 16  V     0     0.75  0.75
 17  V     0.25  0     0
 18  V     0.25  0     0.25
 19  V     0.25  0     0.5
 20  V     0.25  0     0.75
 21  V     0.25  0.25  0
 22  V     0.25  0.25  0.25
 23  V     0.25  0.25  0.5
 24  V     0.25  0.25  0.75
 25  V     0.25  0.5   0
 26  V     0.25  0.5   0.25
 27  V     0.25  0.5   0.5


In [63]:
# create start structure from the entries
create_start_structures(entries, 5, '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64/Start_Structures', cutoff_distance=1.25, supercell_scheme=False)

Processing entry 1/7...
Found target atoms for entry 1: [PeriodicSite: V1 (V) (0.0, 0.0, 2.607) [0.0, 0.0, 0.25], PeriodicSite: V56 (V) (4.915, 4.257, -4.345) [0.75, 0.5, 0.0], PeriodicSite: Cr51 (Cr) (7.373, 0.0, 5.213) [0.75, 0.0, 0.75], PeriodicSite: V12 (V) (-3.686, 6.385, -2.607) [0.0, 0.75, 0.0], PeriodicSite: V15 (V) (-3.686, 6.385, 5.213) [0.0, 0.75, 0.75]]
Written start structure to ../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64/Start_Structures/supercell_gen0_compTi3V58Cr3_vac_site0_start.vasp
Written start structure to ../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64/Start_Structures/supercell_gen0_compTi3V58Cr3_vac_site1_start.vasp
Written start structure to ../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64/Start_Structures/supercell_gen0_compTi3V58Cr3_vac_site2_start.vasp
Written start structure to ../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64/Start_Structures/supercell_gen0_compTi3V58Cr3_vac_site3_start.vasp
Written start structure to ../Vi

In [64]:
computed_entries = json.load(open('../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64/Start_Structures/computed_entries.json'))
print(computed_entries[0]['data'])
print(computed_entries[0]['structure'])
print(type(computed_entries[0]))

{'generation': 0, 'comp': 'Ti3V58Cr3', 'struct': 'Fixed_64', 'vac_index': 4, 'vac_comp': {'@module': 'pymatgen.core.periodic_table', '@class': 'Element', 'element': 'V', '@version': None}, 'perfect_structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattice': {'matrix': [[9.830618833167076, 0.0, -3.475648619229503], [-4.915309414949067, 8.513565645388086, -3.475648619229503], [0.0, 0.0, 10.42694586]], 'pbc': [True, True, True], 'a': 10.42694586, 'b': 10.42694586, 'c': 10.42694586, 'alpha': 109.47122063, 'beta': 109.47122063, 'gamma': 109.47122063, 'volume': 872.6688317522414}, 'properties': {}, 'sites': [{'species': [{'element': 'Ti', 'occu': 1.0}], 'abc': [0.5, 0.75, 0.0], 'xyz': [1.2288273553717382, 6.385174234041065, -4.344560774036879], 'properties': {}, 'label': 'Ti44'}, {'species': [{'element': 'Ti', 'occu': 1.0}], 'abc': [0.75, 0.0, 0.0], 'xyz': [7.372964124875307, 0.0, -2.606736464422127], 'properties': {}, 'label': 'Ti48'}, {'species': [{'

## Relax Each Vacancy Structure

In [12]:
from pymatgen.entries.computed_entries import ComputedStructureEntry
# now lets make the computed entries from chgnet potential 
from chgnet.model.model import CHGNet 
from chgnet.model.dynamics import CHGNetCalculator 
from chgnet.model import StructOptimizer

def create_computed_entry_from_chgnet(structure, energy):
    composition = structure.composition
    return ComputedStructureEntry(composition=composition, energy = energy, structure=structure)

def relax_structures(supercells, potential_path,device='cpu',verbose=False,relax_cell=True):
    vac_pot = CHGNet.from_file(potential_path, use_device=device)
    entries = []
    relaxer = StructOptimizer(vac_pot, use_device=device)

    # If vcrti_generated_supercells is a dictionary, convert it to a list
    if isinstance(supercells, dict):
        supercells = list(supercells.values())
    print(len(supercells))
    for i, supercell in enumerate(supercells):
        if isinstance(supercell, dict):
            supercell = Structure.from_dict(supercell)
        relaxed_supercell = relaxer.relax(atoms=supercell, fmax=0.05, relax_cell=relax_cell, verbose=verbose)
        print("Finished Volumetric Relaxing Structure: ", i)
        final_result = relaxer.relax(atoms=relaxed_supercell['final_structure'], fmax=0.05, relax_cell=False, verbose=verbose)
        print("Final Energy: ", final_result['trajectory'].energies[-1])

        # make the computed entry
        entry = create_computed_entry_from_chgnet(supercell, final_result['trajectory'].energies[-1])
        entries.append(entry.as_dict())

    return entries

In [ ]:
from nequip.ase import NequIPCalculator
species = {
            "C": "NequIPTypeNameForCarbon",
            "H": "NequIPTypeNameForHydrogen",
        }
def allegro_relaxer(atoms, potential_path, species, device='cpu', verbose=False, relax_cell=True):
    atoms.calc = NequIPCalculator.from_deployed_model(
        model_path=potential_path,
        species_to_type_name = species
    )
    
    # need to make our own relaxer for allegro 
    relaxed_supercell = relaxer.relax(atoms=atoms, fmax=0.05, relax_cell=relax_cell, verbose=verbose)
    print("Finished Volumetric Relaxing Structure: ", i)
    final_result = relaxer.relax(atoms=relaxed_supercell['final_structure'], fmax=0.05, relax_cell=False, verbose=verbose)
    print("Final Energy: ", final_result['trajectory'].energies[-1])


    return atoms

In [20]:
structure_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_512/Start_Structures'

# get the structures from each .vasp file in the directory
start_structures = [Structure.from_file(os.path.join(structure_path, f)) for f in os.listdir(structure_path) if f.endswith('.vasp')]

# load chgnet 
pot_path = '../Potentials/Vacancy_Train_Results/bestF_epoch89_e2_f28_s55_mNA.pth.tar'

relaxed_entries = relax_structures([start_structures[0]], pot_path, device='mps', verbose=True)

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on mps
1


KeyboardInterrupt: 

## Create End Points from relaxed Vacancy

In [73]:
# load in the relaxed entries 
entry_path = '../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_64/Start_Structures/relaxed_entries.json'
with open(entry_path, 'r') as file:
    relaxed_entries = json.load(file)


In [79]:
print(relaxed_entries[0][0])

{'@module': 'pymatgen.entries.computed_entries', '@class': 'ComputedStructureEntry', 'energy': 0, 'composition': {'Ti': 3.0, 'V': 57.0, 'Cr': 3.0}, 'entry_id': None, 'correction': 0.0, 'energy_adjustments': [], 'parameters': {}, 'data': {'generation': 0, 'comp': 'Ti3V58Cr3', 'struct': 'Fixed_64', 'vac_index': 4, 'vac_comp': {'@module': 'pymatgen.core.periodic_table', '@class': 'Element', 'element': 'V', '@version': None}, 'perfect_structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0.0, 'lattice': {'matrix': [[9.830618833167076, 0.0, -3.475648619229503], [-4.915309414949067, 8.513565645388086, -3.475648619229503], [0.0, 0.0, 10.42694586]], 'pbc': [True, True, True], 'a': 10.42694586, 'b': 10.42694586, 'c': 10.42694586, 'alpha': 109.47122063, 'beta': 109.47122063, 'gamma': 109.47122063, 'volume': 872.6688317522414}, 'properties': {}, 'sites': [{'species': [{'element': 'Ti', 'occu': 1.0}], 'abc': [0.5, 0.75, 0.0], 'xyz': [1.2288273553717382, 6.385174234041

In [19]:
# save the relaxed structure to a .vasp file 
for i, entry in enumerate(relaxed_entries):
    structure = Structure.from_dict(entry['structure'])
    Poscar(structure).write_file(f'../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_512/Start_Structures_Relaxed/relaxed_structure_{i}.vasp')

# Interpolate Barriers and run the models

In [36]:
relaxed_entries = json.load(open('../Visualization/Job_Structures/Pre_VASP/VCrTi_Fixed_512/debug_relaxed_entries.json'))


In [46]:
start = relaxed_entries[0][0]['structure']
end = relaxed_entries[0][1]['structure']

In [47]:
# interpolate the structures and use shared calculator 
images = [start]
images += [start.copy() for i in range(3)]
images += [end]

In [ ]:
from ase import io
from ase.mep import DyNEB
neb = DyNEB(images,fmax=0.02, dynamic_relaxation=True, scale_fmax=1.,allow_shared_calculator=True)
# Interpolate linearly the potisions of the three middle images:
neb.interpolate()
# Set calculators:
for image in images[1:4]:
    image.calc = calc
# Optimize:
optimizer = LBFGS(neb, trajectory='test.traj')
optimizer.run(fmax=0.02)

# Analyze the Barrier Statistics Results 

In [48]:
v_data = [{'V' : 38, 'Cr' : 19, 'Ti' : 6, 'Mean' : 0.72, 'Std' : 0.16},
 {'V' : 48, 'Cr' : 8, 'Ti' : 7, 'Mean' : 0.61, 'Std' : 0.05},
 {'V' : 38, 'Cr' : 6, 'Ti' : 19, 'Mean' : 0.28, 'Std' : 0.20},
 {'V' : 58, 'Cr' : 3, 'Ti' : 3, 'Mean' : 0.39, 'Std' : 0.05}]

In [80]:
import plotly.figure_factory as ff

# Convert number of atoms to atomic percentages
for data in v_data:
    total_atoms = data['V'] + data['Cr'] + data['Ti']
    data['V'] = data['V'] / total_atoms
    data['Cr'] = data['Cr'] / total_atoms
    data['Ti'] = data['Ti'] / total_atoms

# Create lists of atomic percentages and mean values
V_percentages = [data['V'] for data in v_data]
Cr_percentages = [data['Cr'] for data in v_data]
Ti_percentages = [data['Ti'] for data in v_data]
mean_values = [data['Mean'] for data in v_data]
print(V_percentages)
print(Cr_percentages)
print(Ti_percentages)

import numpy as np

# Convert mean_values to a NumPy array
mean_values = np.array(mean_values)

# Create the ternary plot
fig = ff.create_ternary_contour([V_percentages, Cr_percentages, Ti_percentages], mean_values, pole_labels=['V', 'Cr', 'Ti'])
fig.show()

[0.6031746031746033, 0.7619047619047619, 0.6031746031746031, 0.90625]
[0.3015873015873016, 0.12698412698412698, 0.09523809523809523, 0.046875]
[0.09523809523809523, 0.1111111111111111, 0.30158730158730157, 0.046875]


In [54]:
import plotly.graph_objects as go

# Create the ternary plot
fig = go.Figure(go.Scatterternary({
    'mode': 'markers',
    'a': V_percentages,
    'b': Cr_percentages,
    'c': Ti_percentages,
    'marker': {
        'symbol': 100,
        'color': mean_values,
        'cmax': 1,
        'cmin': 0,
        'colorscale': 'Viridis',
        'colorbar': {'title': 'Mean Value'},
        'line': {'width': 2},
        'size': 10  # Increase the size of the data points
    },
    'text': mean_values,
    'hoverinfo': 'text'
}))

# Rest of the code...

# Set the labels for the axes
fig.update_layout({
    'ternary': {
        'sum': 1,
        'aaxis': {'title': 'V', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
        'baxis': {'title': 'Cr', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
        'caxis': {'title': 'Ti', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'}
    },
    'showlegend': False,
    'autosize': False,
    'width': 800,
    'height': 800,
    'margin': {'b': 0, 'l': 0, 'r': 0, 't': 0},
    'paper_bgcolor': 'rgba(0,0,0,0)',
    'plot_bgcolor': 'rgba(0,0,0,0)'
})

fig.show()

In [53]:
import plotly.graph_objects as go

# Convert number of atoms to atomic percentages
for data in v_data:
    total_atoms = data['V'] + data['Cr'] + data['Ti']
    data['V'] = data['V'] / total_atoms
    data['Cr'] = data['Cr'] / total_atoms
    data['Ti'] = data['Ti'] / total_atoms

# Create lists of atomic percentages and mean values
V_percentages = [data['V'] for data in v_data]
Cr_percentages = [data['Cr'] for data in v_data]
Ti_percentages = [data['Ti'] for data in v_data]
mean_values = [data['Mean'] for data in v_data]

# Convert mean_values to a NumPy array
mean_values = np.array(mean_values)

# Create the ternary plot
# Create the ternary plot
fig = go.Figure(go.Scatterternary({
    'mode': 'markers',
    'a': V_percentages,
    'b': Cr_percentages,
    'c': Ti_percentages,
    'marker': {
        'symbol': 100,
        'color': mean_values,
        'cmax': 1,
        'cmin': 0,
        'colorscale': 'Greys',
        'colorbar': {'title': 'Mean Value'},
        'line': {'width': 2},
        'size': 10  # Increase the size of the data points
    },
    'text': mean_values,
    'hoverinfo': 'text'
}))

# Rest of the code...

# Set the labels for the axes
fig.update_layout({
    'ternary': {
        'sum': 1,
        'aaxis': {'title': 'V', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
        'baxis': {'title': 'Cr', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'},
        'caxis': {'title': 'Ti', 'min': 0.01, 'linewidth': 2, 'ticks': 'outside'}
    },
    'showlegend': False,
    'autosize': False,
    'width': 800,
    'height': 800,
    'margin': {'b': 0, 'l': 0, 'r': 0, 't': 0},
    'paper_bgcolor': 'rgba(0,0,0,0)',
    'plot_bgcolor': 'rgba(0,0,0,0)'
})

fig.show()

In [82]:
# test 

from test_VASPDataParser import * 

parser = VASPDataParser()
data = parser.parse_directory('../Visualization/Job_Structures/Post_VASP/VCrTiWZr_Summit/Vacancies/gen_0_4')


In [90]:
print(data['supercell_gen4_comp51_struct2_vac_site1_start']['structures'][0].as_dict())

{'lattice': [[9.915129, -0.013801, -3.506631], [-4.979905, 8.620921, -3.536122], [0.023182, 0.004032, 10.585464]], 'species': ['Zr', 'Zr', 'Zr', 'Ti', 'Ti', 'Ti', 'Ti', 'Ti', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'V', 'Cr', 'Cr', 'W', 'W'], 'coords': [[0.99456765, 0.49789007, 0.49699113], [0.23325726, 0.50021526, 0.24527806], [0.27748609, 0.49376952, 0.52054139], [0.25257959, 0.00742405, 0.50248465], [0.25345651, 0.49968118, 0.99080777], [0.25461289, 0.75998857, 0.50719779], [0.50713062, 0.49781728, 0.24220648], [0.74851012, 0.25304592, 0.74767697], [0.99681123, 0.99294792, 0.00083195], [0.99655358, 0.99513453, 0.24844063], [0.99327731, 9.808e-05, 0.49820208], [0.0027823, 0.00192605, 0.75091092], [0.99001054, 0.23754578, 0.98629994], [0.98032947, 0.22433855, 0.23194316], [0.991

In [92]:
struct_data = data['supercell_gen4_comp51_struct2_vac_site1_start']['structures'][0].as_dict()
lattice = struct_data['lattice']
species = struct_data['species']
coords = struct_data['coords']
coords_are_cartesian = struct_data['coords_are_cartesian']
Structure(lattice=lattice,
        species=species,
        coords=coords,
        coords_are_cartesian=coords_are_cartesian
        )

Structure Summary
Lattice
    abc : 10.51695937637885 10.565220844646362 10.585490151969534
 angles : 109.59778680389107 109.35142365558409 109.50267385764052
 volume : 905.0056567283126
      A : 9.915129 -0.013801 -3.506631
      B : -4.979905 8.620921 -3.536122
      C : 0.023182 0.004032 10.585464
    pbc : True True True
PeriodicSite: Zr (7.393, 4.281, 0.0127) [0.9946, 0.4979, 0.497]
PeriodicSite: Zr (-0.1726, 4.31, 0.009613) [0.2333, 0.5002, 0.2453]
PeriodicSite: Zr (0.3045, 4.255, 2.791) [0.2775, 0.4938, 0.5205]
PeriodicSite: Ti (2.479, 0.06254, 4.407) [0.2526, 0.007424, 0.5025]
PeriodicSite: Ti (0.04766, 4.308, 7.832) [0.2535, 0.4997, 0.9908]
PeriodicSite: Ti (-1.248, 6.55, 1.789) [0.2546, 0.76, 0.5072]
PeriodicSite: Ti (2.555, 4.286, -0.9748) [0.5071, 0.4978, 0.2422]
PeriodicSite: Ti (6.179, 2.174, 4.395) [0.7485, 0.253, 0.7477]
PeriodicSite: V (4.939, 8.546, -6.998) [0.9968, 0.9929, 0.000832]
PeriodicSite: V (4.931, 8.566, -4.384) [0.9966, 0.9951, 0.2484]
PeriodicSite: V (9.8

In [ ]:
with open('test_parsed_data.json', 'w') as outfile:
    json.dump(data, outfile)
